In [ ]:
from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = "%Y-%m-%d %H:%M"

def add_node_reservation(reservation_list, count=1, node_type="compute_haswell"):
    reservation_list.append({
        "resource_type": "physical:host",
        "resource_properties": json.dumps(["==", "$node_type", node_type]),
        "hypervisor_properties": "",
        "min": count,
        "max": count
    })

def add_network_reservation(reservation_list, 
                            network_name,
                            network_description="", 
                            of_controller_ip=None, 
                            of_controller_port=None, 
                            vswitch_name=None, 
                            physical_network="physnet1"):

    if of_controller_ip != None and of_controller_port != None:
        description = description + 'OFController=' + of_controller_ip + ':' + of_controller_port 
        
    if vswitch_name != None and of_controller_ip != None and of_controller_port != None:
        description = description + ','
    
    if vswitch_name != None:
        description = description + 'VSwitchName=' + vswitch_name

    reservation_list.append({
        "resource_type": "network",
        "network_name": network_name,
        "network_description": network_description,
        "resource_properties": json.dumps(["==", "$physical_network", physical_network]),
        "network_properties": ""
    })
    
def add_fip_reservation(reservation_list, count=1):
    #Get public network id (needed to reserve networks)
    public_network_id = get_public_network(chi.neutron())

    reservation_list.append({
        "resource_type": "virtual:floatingip",
        "network_id": public_network_id,
        "amount": count
    })    
    
    
def reserve_multiple_resources(lease_name):
    # Set start/end date for lease
    # Start one minute into future to avoid Blazar thinking lease is in past
    # due to rounding to closest minute.
    start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
    end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)
    
    # Build list of reservations (in this case there is only one reservation)
    reservation_list = []
    add_node_reservation(reservation_list, count=1, node_type="compute_haswell")
    add_network_reservation(reservation_list, network_name=lease_name+"Network")
    add_fip_reservation(reservation_list, count=1)
    
    # Create the lease
    lease = chi.blazar().lease.create(name=lease_name, 
                                start=start_date,
                                end=end_date,
                                reservations=reservation_list, events=[])

## Examples: 
**Important**: Examples below import API methods from chi library and does not use refernece code above!

#### Reserve Mulitpule Resources

In [ ]:
import json
import os
import chi

from chi.reservation_api_examples import *

#Config with your project and site
chi.set('project_name', 'CH-816532') # Replace with your project name
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'


# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
lease_name = username+'Lease'

#Get a network by name
lease = reserve_multiple_resources(lease_name)
    
#Get the lease by name
lease = get_lease_by_name(lease_name)
    
#Print the lease info
print(json.dumps(lease, indent=2))


